In [1]:
import logging, urllib3

# Suppress all urllib3 warnings and errors
#logging.getLogger("urllib3").setLevel(logging.CRITICAL)

# Also, disable specific SSL warnings if using insecure connections
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from dotenv import load_dotenv

In [3]:
env_loaded = load_dotenv('../../.env-local')
env_loaded

True

In [4]:
from datadocai.models import CurrentTable

TRINO_CATALOG = 'postgres'
TRINO_SCHEMA = 'public'
TRINO_TABLE = 'house_pricing'

# construct the table you want to analyse
ct = CurrentTable(trino_catalog=TRINO_CATALOG,
                  trino_schema=TRINO_SCHEMA,
                  trino_table=TRINO_TABLE)

/home/jouvance/projects/DataDocAi/api/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [5]:
import os
from datadocai.database import DatabaseClient

# connect to trino
dc = DatabaseClient(host=os.getenv('TRINO_HOST'),
                    port=os.getenv('TRINO_PORT'),
                    user=os.getenv('TRINO_USER'),
                    password=os.getenv('TRINO_PASSWORD'))

LOCAL connection


In [6]:
from langchain_openai.chat_models import AzureChatOpenAI
from httpx import Client

llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    openai_api_version="2024-05-01-preview",
    streaming=True,
    verbose=True,
    http_client=Client(verify=False)
)

In [7]:
from datadocai.metadata import TableMetadataManager
from datadocai.metadata.exporter.json import MetadataJsonExporter

metadata_exporter = MetadataJsonExporter(current_table=ct)

tmm = TableMetadataManager(current_table=ct, database_client=dc, llm=llm, metadata_exporter=metadata_exporter)

[Errno 17] File exists: 'outputs/postgres/public'


In [12]:
# launch the process
result, exporter = tmm.process()

AttributeError: 'TableMetadataManager' object has no attribute 'process'